Reads in a geopackage and returns a metadata csv

In [1]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
from shapely import box
from convertbng.util import convert_bng, convert_lonlat


In [2]:
egpkg = 'mra/England/Public Sector England/England_GPKG/England_GPKG.gpkg'
sgpkg = 'mra/Scotland/Public Sector Scotland/Scotland_GPKG/Scotland_GPKG.gpkg'
wgpkg = 'mra/Wales/Public Sector Wales/Wales_GPKG/Wales_GPKG.gpkg'

In [7]:
pkg_layers = gpd.list_layers(egpkg)
layer_list = pkg_layers['name'].values.tolist()
# layer_list.remove('England_GPKG')
layer_list

C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D LineString' is converted to 'LineString Z'
  return ogr_lis

['Coalfield_Consultation_Area',
 'Mine_Entry',
 'Probable_Working',
 'Geological_Disturbance',
 'Outcrops',
 'Legal_Notice',
 'Licence_Area_of_Responsibility',
 'Licence_Area',
 'Parent_Phase',
 'In_Seam_Level_Contour',
 'In_Seam_Level',
 'Underground_Working',
 'Unlicensed_Opencast',
 'Spine_Roadways',
 'Working_Dates',
 'Licensed_Roadways',
 'Court_Order',
 'Court_Order_Amendment',
 'Licence_Table',
 'Seam_Table',
 'Shallow_Working',
 'England_GPKG']

In [8]:
lbounds = list()
lgeom = list()
lgt = list()
gtlist = list()
for i in layer_list: 
    lay = gpd.read_file(egpkg, layer=i)
    layer_bounds = lay.geometry.total_bounds
    lbounds.append(layer_bounds)
    geom = box(*lay.geometry.total_bounds)
    lgeom.append(geom)
    gt = lay.geom_type.unique()
    gtlist.append(gt)
lpd = pd.DataFrame({'name':layer_list,'bounds':lbounds,'geometry':lgeom,'geomtype':gtlist})
lpd.head()

C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
C:\Users\joken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCach

,name,bounds,geometry,geomtype
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]"
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99000000022, 633740 65...",[Point]
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.75999999978, 489617 65...","[Polygon, None]"
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString]
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38000000082, 453883 65...",[LineString]


In [42]:
lgpd.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [10]:
lgpd = gpd.GeoDataFrame(lpd, crs="EPSG:27700")
lgpd['epsg4326'] = lgpd['geometry'].to_crs("EPSG:4326")
lgpd.head()

,name,bounds,geometry,geomtype,epsg4326
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]","POLYGON ((1.55745 50.9432, 1.98283 55.75521, -..."
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99, 633740 658335.54, ...",[Point],"POLYGON ((1.32843 50.97827, 1.72418 55.76185, ..."
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.76, 489617 651236, 244...","[Polygon, None]","POLYGON ((-0.72368 51.0204, -0.57399 55.74622,..."
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString],"POLYGON ((-0.95617 51.21754, -0.83815 55.76483..."
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38, 453883 652643, 245...",[LineString],"POLYGON ((-1.23319 51.01824, -1.14285 55.76414..."


In [12]:
lgpd = lgpd.loc[lgpd['geometry'].is_valid, :]
lgpd

,name,bounds,geometry,geomtype,epsg4326
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]","POLYGON ((1.55745 50.9432, 1.98283 55.75521, -..."
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99, 633740 658335.54, ...",[Point],"POLYGON ((1.32843 50.97827, 1.72418 55.76185, ..."
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.76, 489617 651236, 244...","[Polygon, None]","POLYGON ((-0.72368 51.0204, -0.57399 55.74622,..."
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString],"POLYGON ((-0.95617 51.21754, -0.83815 55.76483..."
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38, 453883 652643, 245...",[LineString],"POLYGON ((-1.23319 51.01824, -1.14285 55.76414..."
5,Legal_Notice,"[297155.0, 145979.0, 637738.0, 647834.0]","POLYGON ((637738 145979, 637738 647834, 297155...",[Polygon],"POLYGON ((1.39897 51.16363, 1.77864 55.66575, ..."
6,Licence_Area_of_Responsibility,"[305027.0, 299539.3100000005, 466462.200000000...","POLYGON ((466462.2 299539.31, 466462.2 582291....",[Polygon],"POLYGON ((-1.02038 52.5896, -0.95918 55.13055,..."
7,Licence_Area,"[280001.3099999996, 134709.0, 643224.700000000...","POLYGON ((643224.7 134709, 643224.7 642873, 28...",[Polygon],"POLYGON ((1.4697 51.06017, 1.86128 55.61857, -..."
8,Parent_Phase,"[246967.8200000003, 126117.5, 637099.0, 651584...","POLYGON ((637099 126117.5, 637099 651584.31, 2...",[Polygon],"POLYGON ((1.37683 50.9856, 1.77174 55.69969, -..."
9,In_Seam_Level_Contour,"[294825.0, 147363.0, 637127.0, 648541.0]","POLYGON ((637127 147363, 637127 648541, 294825...",[LineString],"POLYGON ((1.39116 51.1763, 1.76956 55.67239, -..."


In [18]:
bounding_boxes = lgpd.geometry.bounds
layergpd = lgpd.join(bounding_boxes)
layergpd

,name,bounds,geometry,geomtype,epsg4326,minx,miny,maxx,maxy
0,Coalfield_Consultation_Area,"[240000.0, 122000.0, 650000.0, 658500.0]","POLYGON ((650000 122000, 650000 658500, 240000...","[Polygon, None]","POLYGON ((1.55745 50.9432, 1.98283 55.75521, -...",240000.000,122000.0000,650000.00,658500.0000
1,Mine_Entry,"[245082.8700000001, 125147.99000000022, 633740...","POLYGON ((633740 125147.99, 633740 658335.54, ...",[Point],"POLYGON ((1.32843 50.97827, 1.72418 55.76185, ...",245082.870,125147.9900,633740.00,658335.5395
2,Probable_Working,"[244983.41999999993, 125333.75999999978, 48961...","POLYGON ((489617 125333.76, 489617 651236, 244...","[Polygon, None]","POLYGON ((-0.72368 51.0204, -0.57399 55.74622,...",244983.420,125333.7600,489617.00,651236.0000
3,Geological_Disturbance,"[296168.0, 147000.0, 473000.0, 652998.0]","POLYGON ((473000 147000, 473000 652998, 296168...",[LineString],"POLYGON ((-0.95617 51.21754, -0.83815 55.76483...",296168.000,147000.0000,473000.00,652998.0000
4,Outcrops,"[245408.7000000002, 124597.38000000082, 453883...","POLYGON ((453883 124597.38, 453883 652643, 245...",[LineString],"POLYGON ((-1.23319 51.01824, -1.14285 55.76414...",245408.700,124597.3800,453883.00,652643.0000
5,Legal_Notice,"[297155.0, 145979.0, 637738.0, 647834.0]","POLYGON ((637738 145979, 637738 647834, 297155...",[Polygon],"POLYGON ((1.39897 51.16363, 1.77864 55.66575, ...",297155.000,145979.0000,637738.00,647834.0000
6,Licence_Area_of_Responsibility,"[305027.0, 299539.3100000005, 466462.200000000...","POLYGON ((466462.2 299539.31, 466462.2 582291....",[Polygon],"POLYGON ((-1.02038 52.5896, -0.95918 55.13055,...",305027.000,299539.3100,466462.20,582291.9284
7,Licence_Area,"[280001.3099999996, 134709.0, 643224.700000000...","POLYGON ((643224.7 134709, 643224.7 642873, 28...",[Polygon],"POLYGON ((1.4697 51.06017, 1.86128 55.61857, -...",280001.310,134709.0000,643224.70,642873.0000
8,Parent_Phase,"[246967.8200000003, 126117.5, 637099.0, 651584...","POLYGON ((637099 126117.5, 637099 651584.31, 2...",[Polygon],"POLYGON ((1.37683 50.9856, 1.77174 55.69969, -...",246967.820,126117.5000,637099.00,651584.3100
9,In_Seam_Level_Contour,"[294825.0, 147363.0, 637127.0, 648541.0]","POLYGON ((637127 147363, 637127 648541, 294825...",[LineString],"POLYGON ((1.39116 51.1763, 1.76956 55.67239, -...",294825.000,147363.0000,637127.00,648541.0000


In [ ]:
layergpd.to_csv('gpkg_meta.csv')